<a href="https://colab.research.google.com/github/SomdeepAcharyya/Fake-News-classifiaction/blob/main/Credsat_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CredSaT : Credibility ranking of users in big social data incorporating semantic analysis and temporal factor

In [ ]:
from google.colab import files
import io
import pandas as pd
import spacy
from textblob import TextBlob
from spacy.lang.en import English
nlp = English()
import csv
from itertools import chain
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
nltk.download('wordnet')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nlp = spacy.load('en')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
spc = spacy.load('en_core_web_sm')
import http.client
import urllib.parse
from urllib.request import urlopen
import math
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.ESCAPE_CHAR, p.OPT.RESERVED)
import datetime
from time import strptime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = r'/content/drive/MyDrive/Colab Notebooks/boston_bombings_score.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  df = pd.read_csv(infile)

In [ ]:
df_tweets = df

In [ ]:
class my_dictionary(dict):
  
    # __init__ function
    def __init__(self):
        self = dict()
          
    # Function to add key:value
    def add(self, key, value):
        self[key] = value
        
def checkKey(dict, key): 
        if not key in dict.keys():
            return 0
        else:
            return 1
def check(a):
    if a == None:
        return 0
    else:
        return 1
def Find_URL(string):
    # findall() has been used 
    # with valid conditions for urls in string
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    #url = re.findall(r'(https?://\S+)', s)
    url = re.findall(regex,string)
    return [x[0] for x in url]
def unshorten_url(url):
    parsed = urllib.parse.urlparse(url)
    h = http.client.HTTPConnection(parsed.netloc)
    #h = http.clent.HTTPConnection(parsed.netloc)
    h.request('HEAD', parsed.path)
    response = h.getresponse()
    if response.status/100 == 3 and response.getheader('Location'):
        return response.getheader('Location')
    else:
        return url

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2013-boston-marathon-bombing.csv to 2013-boston-marathon-bombing (1).csv


In [ ]:
uploaded = files.upload()

Saving UserMatrix_0510.csv to UserMatrix_0510.csv


In [ ]:
df_user = pd.read_csv('UserMatrix_0510.csv')
df_tweets = pd.read_csv('UserTweet_0510.csv')
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.reset_index(drop=True)
df_user = df_user.dropna()
df_user = df_user.reset_index(drop=True)

In [ ]:
df_tweets.columns

Index(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'screen_name', 'user_default_profile_image', 'description',
       'favourites_count', 'followers_count', 'friends_count', 'listed_count',
       'location', 'user_name', 'user_screen_name', 'statuses_count',
       'time_zone', 'user_urls', 'verified'],
      dtype='object')

In [ ]:
lemmatizer = WordNetLemmatizer()
punctuation = re.compile(r'[-.?&!,:;()|0-9]\/`'"'")
def get_lemma_words(t):
    t = t.lower()
    #a = nlp(t)
    lemmat= " "
    stop_words = set(stopwords.words('english'))  
    word_tokens = word_tokenize(p.clean(t)) 
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    punc_words = []
    for words in filtered_sentence:
        word =  punctuation.sub("", words)
        if len(word) > 0:
          punc_words.append(word)
    for word in punc_words:
        lemmat = lemmat + " " + (lemmatizer.lemmatize(word))
    #print('y')
    return lemmat

In [ ]:
#processing text by preprocessing & lemmatization

processed_text = []
for i in range(len(df_tweets)):
  x = df_tweets['text'][i]
  y = get_lemma_words(x)
  processed_text.append(y)
df_tweets['processed_text'] = processed_text

In [ ]:
df_tweets.to_csv('processed.csv', sep=',', index = False)
files.download('processed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
text = []
for i in range(len(df_tweets)):
  text.append(df_tweets['processed_text'][i])

In [ ]:
# clustering of text into domains

clusters = []
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(text)

true_k = 7
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    rows = []
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        rows.append(' %s' % terms[ind])
        print(' %s' % terms[ind]),
    print
    clusters.append(rows)

c= []
for i in clusters:
    s = ""
    for j in i:
      s = s + " " + j.lower()
    c.append(s)
d = []
for i in range(len(c)):
    input=[c[i]]
    d.append(c[i])

In [ ]:
d

['  720dop  eqg7v8nqed  31butter  badge  btsbutter  freebie  600  php  ph  cassette',
 '  50  unlock  vote  repliesbutteready  bbmasi  challenge  http  retweetsbutteready  hoop  reply',
 '  display  charm  alluring  4ylpdxcmwa  member  group  photo  teaser  butter  gy7ydcisnv',
 '  yx4w1aigej  msno  terminar  viva  faltan  final  comeback  foto  este  al',
 '  http  vote  butteready  2021  bbmasi  bbmas  butter  bts  photo  teaser',
 '  kl7exvfw0d  vinyl  cassette  butter  army  http  amp  ns0zsyh2zy  pe8heoq28q  pdyd9qoaoi',
 '  hottest  stage  thing  jimin  estuvisemos  et3qs0szsghttps  estuvimos  etathis  etaojljurc  etaoctober']

In [ ]:
#finding domain and intersection of words

def intersection(lst1, lst2):
    count = 0
    x = lst1.split()
    y = lst2.split()
    for i in x:
      if i in y:
        count = count + 1
    return count

intersection_words = []
domain = []
for i in range(len(df_tweets)):
    tv = []
    for k in range(len(d)):
      tv.append(intersection(df_tweets['processed_text'][i], d[k]))
    domain.append(tv.index(max(tv)))
    intersection_words.append(max(tv))

df_tweets['domain'] = domain
df_tweets['intersection_words'] = intersection_words

In [ ]:
# polarity & subjectivity 

df_tweets['sentiment'] = 0.000

def get_polarity(a):
  analysis = TextBlob(a).polarity
  return analysis
def get_subjectivity(a):
  analysis = TextBlob(a).subjectivity
  return analysis

sentiment = []

for i in range(len(df_tweets)):
    x = df_tweets['processed_text'][i]
    sentiment.append(get_polarity(x)+get_subjectivity(x))
df_tweets['sentiment']  = sentiment

In [ ]:
df_tweets['sentiment']

0        0.000000
1        0.000000
2        0.366883
3        0.000000
4        0.000000
           ...   
36270    0.000000
36271    0.000000
36272    0.000000
36273    0.000000
36274    0.000000
Name: sentiment, Length: 36275, dtype: float64

In [ ]:
# calculate sct

ud = []
for a in range(len(df_user)):
  rows = [0,0,0,0,0,0,0]
  ud.append(rows)
df_user['sct'] = ud
for i in range(len(df_tweets)):
    ud = [0,0,0,0]
    x = df_tweets['processed_text'][i]
    y = df_tweets['domain'][i]
    z = df_tweets['screen_name'][i]
    #print(z)
    s = df_tweets['sentiment'][i]
    for j in range(len(df_user)):
      if z == df_user['screen_name'][j]:
        s_p = list(df_user['sct'][j])
        s_p[y] = s_p[y]+s
        df_user['sct'][j] = s_p 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df_user['sct']

0                                  [0, 0, 0, 0, 0, 0, 0]
1             [0.0, 3.6996392496392496, 0.0, 0, 0, 0, 0]
2      [0, 7.188528138528138, 0, 0, 3.519696969696969...
3      [1.5138888888888888, 22.83055555555556, 0.0, 0...
4      [1.0, 6.146748251748251, 0, 0, 5.1363636363636...
                             ...                        
890                            [0.0, 0.0, 0, 0, 0, 0, 0]
891                            [0.0, 0.0, 0, 0, 0, 0, 0]
892                            [0.0, 0.0, 0, 0, 0, 0, 0]
893                              [0, 0.0, 0, 0, 0, 0, 0]
894          [0.0, 0.0, 0, 0, 0.06666666666666667, 0, 0]
Name: sct, Length: 895, dtype: object

In [ ]:
# calculate twt_sim

df_user['twt_sim'] = 0
df_user['distinct'] = 0
df_user['intersection'] = 0
for i in range(len(df_tweets)):
  x = df_tweets['screen_name'][i]
  y = df_tweets['intersection_words'][i]
  z = df_tweets['processed_text'][i]
  for j in range(len(df_user)):
      if x == df_user['screen_name'][j]:
        df_user['distinct'][j] = df_user['distinct'][j] + len(z.split(" "))
        df_user['intersection'][j] = df_user['intersection'][j] + y

for i in range(len(df_user)):
  df_user['twt_sim'][i] = (df_user['intersection'][i]/df_user['distinct'][i])

for i in range(len(df_user)):
  if math.isnan(df_user['twt_sim'][i]):
    df_user['twt_sim'][i] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in long_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

In [ ]:
# calculate score matrix

ud = []
for a in range(len(df_user)):
  rows = [0,0,0,0,0,0,0]
  ud.append(rows)
df_user['score_matrix'] = ud

for i in range(len(df_user)):
  x = df_user['twt_sim'][i]
  y = 0
  for j in range(len(df_user['sct'][i])):
    y = x*df_user['sct'][i][j]
    df_user['score_matrix'][i][j] = y

In [ ]:
# calculate idf

df_user['idf'] = 0.000
ud = []
for a in range(len(df_user)):
  rows = [0,0,0,0,0,0,0]
  ud.append(rows)
df_user['no_domains'] = ud
for i in range(len(df_tweets)):
  x = df_tweets['screen_name'][i]
  y = df_tweets['domain'][i]
  z = df_tweets['processed_text'][i]
  for j in range(len(df_user)):
      if x == df_user['screen_name'][j]:
        df_user['no_domains'][j][y] = df_user['no_domains'][j][y]+1 

for i in range(len(df_user)):
  dfu = 0
  for j in range(len(df_user['no_domains'][i])):
    if df_user['no_domains'][i][j] != 0:
      dfu = dfu + 1
  df_user['idf'][i] = math.log(4/dfu) if dfu != 0 else 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# calculate weight matrix

ud = []
for a in range(len(df_user)):
  rows = [0.000,0.000,0.000,0.000, 0.000, 0.000, 0.000]
  ud.append(rows)
df_user['weight_matrix'] = ud

for i in range(len(df_user)):
  x = df_user['idf'][i]
  y = 0
  for j in range(len(df_user['score_matrix'][i])):
    y = x*df_user['score_matrix'][i][j]
    df_user['weight_matrix'][i][j] = y

In [ ]:
df_user['weight_matrix']

0                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1      [0.0, 0.25571321952224885, 0.0, 0.0, 0.0, 0.0,...
2      [0.0, 0.7298888688862222, 0.0, 0.0, 0.35737324...
3      [-0.05292155188771863, -0.7980958439727698, -0...
4      [0.0526144203304142, 0.32340759618272114, 0.0,...
                             ...                        
890                  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
891                  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
892                  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
893                  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
894    [0.0, 0.0, 0.0, 0.0, 0.0016922474850104756, 0....
Name: weight_matrix, Length: 895, dtype: object

In [ ]:
for d in range(7):
  maxi = 0
  for i in range(len(df_user)):
    x = df_user['weight_matrix'][i][d]
    if x>maxi:
        maxi = x
  for j in range(len(df_user)):
    df_user['weight_matrix'][j][d] =  df_user['weight_matrix'][j][d]/maxi

In [ ]:
df_user['weight_matrix']

0                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1      [0.0, 0.019285220494689725, 0.0, 0.0, 0.0, 0.0...
2      [0.0, 0.05504630460399705, 0.0, 0.0, 0.0804539...
3      [-0.06678339095008762, -0.06019029581523527, -...
4      [0.06639581186117334, 0.02439055287674985, 0.0...
                             ...                        
890                  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
891                  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
892                  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
893                  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
894    [0.0, 0.0, 0.0, 0.0, 0.00038096845044689444, 0...
Name: weight_matrix, Length: 895, dtype: object

In [ ]:
# calculating likes matrix

ud = []
for a in range(len(df_user)):
  rows = [0,0,0,0,0,0,0]
  ud.append(rows)
df_user['likes'] = ud

for i in range(len(df_tweets)):
  x = df_tweets['screen_name'][i]
  y = df_tweets['likes'][i]
  z = df_tweets['domain'][i]
  for j in range(len(df_user)):
    if x == df_user['screen_name'][j]:
      df_user['likes'][j][z] = df_user['likes'][j][z]+y

In [ ]:
ud = []
for a in range(len(df_user)):
  rows = [0.000,0.000,0.000,0.000, 0.000,0.000,0.000]
  ud.append(rows)
df_user['likes_norm'] = ud
for d in range(4):
  max = 0
  for i in range(len(df_user)):
    x = df_user['likes'][i][d]
    if x>max:
        max = x
  for j in range(len(df_user)):
    df_user['likes_norm'][j][d] =  df_user['likes'][j][d]/(max if max !=0 else 1)

In [ ]:
df_user['likes_norm']

0                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1      [7.016865502390396e-05, 1.8935737446790622e-05...
2      [0.0, 0.00023512205007724583, 0.0, 0.0, 0.0, 0...
3      [0.0002654632333421821, 0.00019809890984003116...
4      [2.126322879512241e-06, 1.8300195764429054e-06...
                             ...                        
890    [1.4884260156585688e-05, 2.014440153913896e-07...
891    [0.00013264983502187904, 1.6938320730797123e-0...
892    [1.3085063873921483e-06, 3.347942791011827e-07...
893    [0.0, 6.241927237479677e-08, 0.0, 0.0, 0.0, 0....
894                  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Name: likes_norm, Length: 895, dtype: object

In [ ]:
# calculating retweet matrix
ud = []
for a in range(len(df_user)):
  rows = [0,0,0,0,0,0,0]
  ud.append(rows)
df_user['retweets'] = ud

for i in range(len(df_tweets)):
  x = df_tweets['screen_name'][i]
  y = df_tweets['text'][i]
  z = df_tweets['domain'][i]
  if '@' in y:
    for j in range(len(df_user)):
        if x == df_user['screen_name'][j]:
          df_user['retweets'][j][z] = df_user['retweets'][j][z] + 1

In [ ]:
ud = []
for a in range(len(df_user)):
  rows = [0.000,0.000,0.000,0.000,0.000,0.000,0.000]
  ud.append(rows)
df_user['retweets_norm'] = ud
for d in range(4):
  max = 0
  for i in range(len(df_user)):
    x = df_user['retweets'][i][d]
    if x>max:
        max = x
  for j in range(len(df_user)):
    df_user['retweets_norm'][j][d] =  df_user['retweets'][j][d]/(max if max !=0 else 1)

In [ ]:
df_user['retweets_norm']

0                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1      [0.10416666666666667, 0.06521739130434782, 0.0...
2                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
3      [0.0625, 0.2391304347826087, 0.0, 0.0, 0.0, 0....
4      [0.0, 0.010869565217391304, 0.0, 0.0, 0.0, 0.0...
                             ...                        
890    [0.14583333333333334, 0.010869565217391304, 0....
891    [0.2708333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...
892    [0.020833333333333332, 0.010869565217391304, 0...
893    [0.0, 0.010869565217391304, 0.0, 0.0, 0.0, 0.0...
894    [0.020833333333333332, 0.0, 0.0, 0.0, 0.0, 0.0...
Name: retweets_norm, Length: 895, dtype: object

In [ ]:
# calculating followers-friends

df_user['ffr'] = 0.000

for i in range(len(df_user)):
  x = df_user['favourites_count'][i]
  y = df_user['friends_count'][i]
  z = df_user['created_at'][i]
  a = datetime.date.today()
  year = int(z.split(" ")[5])
  mon = (strptime(z.split(" ")[1],'%b').tm_mon)
  d = int(z.split(" ")[2])
  std = datetime.date(year, mon, d)
  delta = a-std
  df_user['ffr'][i] = ((x - y)/delta.days) if (x-y) != 0 else (1/delta.days)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
max = 0
min = 1000000000
df_user['ffr_norm'] = 0.000
for i in range(len(df_user)):
    x = df_user['ffr'][i]
    if x>max:
        max = x
    if x<min:
        min = x
for j in range(len(df_user)):
    df_user['ffr_norm'][j]=  (df_user['ffr'][j]-min)/(max-min)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
df_user['ffr_norm']

0      0.308292
1      0.063407
2      0.059203
3      0.057795
4      0.059248
         ...   
890    0.283214
891    0.143304
892    0.087143
893    0.096739
894    0.111069
Name: ffr_norm, Length: 895, dtype: float64

In [ ]:
ud = []
for a in range(len(df_user)):
  rows = [0.000,0.000,0.000,0.000,0.000,0.000,0.000]
  ud.append(rows)
df_user['credibility'] = ud

for i in range(len(df_user)):
  for j in range(7):
    df_user['credibility'][i][j] = 0.25*(df_user['ffr'][i]+df_user['weight_matrix'][i][j]+df_user['retweets_norm'][i][j]+df_user['likes_norm'][i][j])


In [ ]:
df_user['credibility']

0      [41.35479323308271, 41.35479323308271, 41.3547...
1      [0.7643480437818789, 0.7594192218355774, 0.738...
2      [0.04109964881474978, 0.05492000547826835, 0.0...
3      [-0.193486822998813, -0.14769778160032326, -0....
4      [0.06513606991186686, 0.057352072394283055, 0....
                             ...                        
890    [37.23174959433448, 37.19800498160146, 37.1952...
891    [14.057788334199584, 13.990047261865513, 13.99...
892    [4.680305185556455, 4.677814000099443, 4.67509...
893    [6.266798418972332, 6.269515825881498, 6.26679...
894    [8.648706091180868, 8.643497757847534, 8.64349...
Name: credibility, Length: 895, dtype: object

In [ ]:
df_user['credibility_total'] = 0.000

for i in range(len(df_user)):
  for j in range(7):
    df_user['credibility_total'][i] = df_user['credibility_total'][i]+df_user['credibility'][i][j]

df_user['credibility_total']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0      289.483553
1        5.215511
2        0.321631
3       -1.315400
4        0.380381
          ...    
890    260.406192
891     97.998070
892     32.733602
893     43.870306
894     60.509788
Name: credibility_total, Length: 895, dtype: float64

In [ ]:
cred_df = df_user[['screen_name', 'credibility_total']]
cred_df.to_csv('credsat_1.csv', sep=',', index = False)
files.download('credsat_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('credsat_0511.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = pd.read_csv("2013-boston-marathon-bombing.csv")

FileNotFoundError: ignored